In [13]:
!ls


output	processed  raw


In [15]:
import pandas as pd

df_raw = pd.read_csv("raw/raw_data.csv")

print("Raw data shape:", df_raw.shape)
df_raw.head()


FileNotFoundError: [Errno 2] No such file or directory: 'raw/raw_data.csv'

In [16]:
import os
os.getcwd()

'/content'

In [17]:
%cd /content


/content


In [18]:
!ls


output	processed  raw	raw_data.csv


In [19]:
import pandas as pd

df_raw = pd.read_csv("raw/raw_data.csv")

print("Raw data shape:", df_raw.shape)
df_raw.head()


FileNotFoundError: [Errno 2] No such file or directory: 'raw/raw_data.csv'

In [20]:
!mv raw_data.csv raw/
!ls raw


In [21]:
!ls raw


raw_data.csv  sample_data


In [22]:
import pandas as pd

df_raw = pd.read_csv("raw/raw_data.csv")
print("Raw data shape:", df_raw.shape)
df_raw.head()


Raw data shape: (1000, 9)


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [32]:
readme_text = """
# Task 14 – ETL Mini Pipeline

## Dataset
Retail Sales Dataset

## Tools
- Python (Google Colab)
- pandas
- SQLite

## ETL Steps
1. Extracted raw CSV into raw layer
2. Cleaned missing values and duplicates
3. Standardized schema and datatypes
4. Created derived business columns
5. Split data into customers, orders, products
6. Loaded outputs into CSV and SQLite
7. Validated row counts and keys

## Outputs
- processed/processed_data.csv
- output/customers.csv
- output/products.csv
- output/orders.csv
- output/database.sqlite
"""

with open("README.md", "w") as f:
    f.write(readme_text)

!ls


output	processed  raw	README.md


In [23]:
df = df_raw.copy()

# remove duplicates
df = df.drop_duplicates()

# check missing values
print(df.isnull().sum())

# drop missing rows (safe here)
df = df.dropna()

print("After cleaning shape:", df.shape)


Transaction ID      0
Date                0
Customer ID         0
Gender              0
Age                 0
Product Category    0
Quantity            0
Price per Unit      0
Total Amount        0
dtype: int64
After cleaning shape: (1000, 9)


In [24]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df.columns


Index(['transaction_id', 'date', 'customer_id', 'gender', 'age',
       'product_category', 'quantity', 'price_per_unit', 'total_amount'],
      dtype='object')

In [25]:
df["date"] = pd.to_datetime(df["date"])
df["age"] = df["age"].astype(int)
df["quantity"] = df["quantity"].astype(int)
df["price_per_unit"] = df["price_per_unit"].astype(float)
df["total_amount"] = df["total_amount"].astype(float)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    1000 non-null   int64         
 1   date              1000 non-null   datetime64[ns]
 2   customer_id       1000 non-null   object        
 3   gender            1000 non-null   object        
 4   age               1000 non-null   int64         
 5   product_category  1000 non-null   object        
 6   quantity          1000 non-null   int64         
 7   price_per_unit    1000 non-null   float64       
 8   total_amount      1000 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 70.4+ KB


In [26]:
# revenue check
df["calculated_amount"] = df["quantity"] * df["price_per_unit"]

# high value transaction flag
df["high_value_txn"] = df["total_amount"] > 1000


In [27]:
df.to_csv("processed/processed_data.csv", index=False)


In [28]:
customers = df[[
    "customer_id", "gender", "age"
]].drop_duplicates()

products = df[[
    "product_category", "price_per_unit"
]].drop_duplicates()

orders = df[[
    "transaction_id", "date", "customer_id",
    "product_category", "quantity", "total_amount"
]]


In [29]:
customers.to_csv("output/customers.csv", index=False)
products.to_csv("output/products.csv", index=False)
orders.to_csv("output/orders.csv", index=False)


In [30]:
import sqlite3

conn = sqlite3.connect("output/database.sqlite")

customers.to_sql("customers", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)

conn.close()


In [31]:
print("Raw rows:", df_raw.shape[0])
print("Processed rows:", df.shape[0])

assert customers["customer_id"].is_unique
assert orders["transaction_id"].is_unique


Raw rows: 1000
Processed rows: 1000


# Task 14 – ETL Mini Pipeline

## Dataset
Retail Sales Dataset

## Tools
- Python (Google Colab)
- pandas
- SQLite

## ETL Steps
1. Extracted raw CSV into raw layer
2. Cleaned missing values and duplicates
3. Standardized schema and datatypes
4. Created derived business columns
5. Split data into customers, orders, products
6. Loaded outputs into CSV and SQLite
7. Validated row counts and keys

## Outputs
- processed/processed_data.csv
- output/customers.csv
- output/products.csv
- output/orders.csv
- output/database.sqlite
